# **Setting Up Environment and Libraries**
In this section, we'll import the necessary libraries and set up the environment for our image classification task.

In [ ]:
import numpy as np
import os
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from google.colab import drive
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# **Data Preprocessing Class**
In this section, we define a class DataPreprocessing to handle the preprocessing of our image data.

In [11]:
class DataPreprocessing():
    def __init__(self, directory_path, target_size=(224, 224), batch_size=32):
        self.directory_path = directory_path
        self.target_size = target_size
        self.batch_size = batch_size

    def generate_datasets(self):
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            validation_split=0.25

        )

        train_generator = train_datagen.flow_from_directory(
            self.directory_path,
            target_size=self.target_size,
            batch_size=self.batch_size,
            class_mode='categorical',
            subset='training',
            color_mode='grayscale',
            seed=42
        )

        validation_generator = train_datagen.flow_from_directory(
            self.directory_path,
            target_size=self.target_size,
            batch_size=self.batch_size,
            class_mode='categorical',
            subset='validation',
            color_mode='grayscale',
            interpolation='bilinear',
            seed=42
        )

        return train_generator, validation_generator

In [ ]:
class ModelCreation():
    def __init__(self, input_shape, num_classes, l2_weight=0.03, l1_weight=0.03):
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.l2_weight = l2_weight
        self.l1_weight = l1_weight

    def create_model(self):
        model = Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=self.input_shape,
                   kernel_regularizer=regularizers.l2(self.l2_weight),
                   activity_regularizer=regularizers.l1(self.l1_weight)),
            MaxPooling2D((2, 2)),
            Conv2D(64, (3, 3), activation='relu',
                   kernel_regularizer=regularizers.l2(self.l2_weight),
                   activity_regularizer=regularizers.l1(self.l1_weight)),
            MaxPooling2D((2, 2)),
            Conv2D(128, (3, 3), activation='relu',
                   kernel_regularizer=regularizers.l2(self.l2_weight),
                   activity_regularizer=regularizers.l1(self.l1_weight)),
            MaxPooling2D((2, 2)),
            Flatten(),
            Dense(128, activation='relu',
                  kernel_regularizer=regularizers.l2(self.l2_weight),
                  activity_regularizer=regularizers.l1(self.l1_weight)),
            Dropout(0.5),
            Dense(self.num_classes, activation='softmax')
        ])
        return model

In [ ]:
class ModelEvaluation:
    def __init__(self):
        pass

    def plot_learning_curves(self, history):
        plt.plot(history.history['accuracy'], label='accuracy')
        plt.plot(history.history['val_accuracy'], label='val_accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.ylim([0, 1])
        plt.legend(loc='lower right')
        plt.show()

        plt.plot(history.history['loss'], label='loss')
        plt.plot(history.history['val_loss'], label='val_loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend(loc='upper right')
        plt.show()

    def visualize_feature_maps(self, model, image):
        # Create a model that outputs the activations of all convolutional layers
        layer_outputs = [layer.output for layer in model.layers if isinstance(layer, tf.keras.layers.Conv2D)]
        activation_model = tf.keras.models.Model(inputs=model.input, outputs=layer_outputs)

        # Get the activations for the input image
        activations = activation_model.predict(np.expand_dims(image, axis=0))

        # Plot the activations for each layer
        for i, activation in enumerate(activations):
            num_features = activation.shape[-1]
            size = activation.shape[1]
            num_cols = min(num_features, 4)
            num_rows = num_features // num_cols

            plt.figure(figsize=(size, size))
            for j in range(num_features):
                ax = plt.subplot(num_rows, num_cols, j + 1)
                ax.set_xticks([])
                ax.set_yticks([])
                ax.imshow(activation[0, :, :, j], cmap='viridis')
            plt.show()

    def plot_roc_curve(self, model, test_data):
        y_true = []
        y_pred = []

        # Predict probabilities for test data
        for x, y in test_data:
            y_true.extend(np.argmax(y, axis=1))
            y_pred.extend(np.argmax(model.predict(x), axis=1))

        fpr = dict()
        tpr = dict()
        roc_auc = dict()

        # Compute ROC curve and ROC area for each class
        for i in range(len(np.unique(y_true))):
            fpr[i], tpr[i], _ = roc_curve(np.array(y_true) == i, np.array(y_pred) == i)
            roc_auc[i] = auc(fpr[i], tpr[i])

        # Plot ROC curve for each class
        plt.figure()
        colors = ['blue', 'red', 'green']  # Add more colors if needed
        for i, color in zip(range(len(np.unique(y_true))), colors):
            plt.plot(fpr[i], tpr[i], color=color, lw=2,
                     label='ROC curve of class {0} (area = {1:0.2f})'
                     ''.format(i, roc_auc[i]))

        plt.plot([0, 1], [0, 1], 'k--', lw=2)
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC) Curve')
        plt.legend(loc="lower right")
        plt.show()

In [12]:
def main():
    data_dir = "/content/drive/MyDrive/AI Pneumonia Detector/chest_xrays"
    input_shape = (224, 224, 1)
    num_classes = 3

    data_preprocessor = DataPreprocessing(data_dir)
    train_data, validation_data = data_preprocessor.generate_datasets()

    model_creator = ModelCreation(input_shape, num_classes)
    model = model_creator.create_model()

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Mount Google Drive
    drive.mount('/content/drive')

    # Specify the directory in Google Drive to save the model file
    model_save_path = "/content/drive/My Drive/AI Pneumonia Detector/best_model2.h5"

    # Configure ModelCheckpoint to save the best model to Google Drive
    checkpoint = ModelCheckpoint(model_save_path, monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    history = model.fit(train_data,
                        epochs=20,
                        validation_data=validation_data,
                        callbacks=[checkpoint, early_stopping])

    evaluation = ModelEvaluation()
    # Validation and Loss Curves plot for train and validation
    evaluation.plot_learning_curves(history)
    # ROC Curve plot for the validation data
    evaluation.plot_roc_curve(model, validation_data)

if __name__ == "__main__":
    main()


Found 1474 images belonging to 3 classes.
Found 490 images belonging to 3 classes.


MessageError: Error: credential propagation was unsuccessful